In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from datetime import datetime
import pandas as pd

def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent / 100)
    height = int(frame.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

columns = ['video file','frame number', 'transition', 'trunk angle', 'trunk acceleration']
log_df = pd.DataFrame(columns=columns)

columns_keypoints = ['video_name', 'frame_number', 'keypoint', 'x_coordinate', 'y_coordinate', 'visibility_score']
keypoints_df = pd.DataFrame(columns=columns_keypoints)

import os

video_folder = "C:/Users/Downloads/20"
results_folder = "C:/Users/Downloads/white"
os.makedirs(results_folder, exist_ok=True)
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    output_video_path = os.path.join(results_folder, f'output_{video_file}')

    model = YOLO('./models/yolov8m-pose.pt')
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    previous_angle = 95
    transition_time_sit_to_stand = 0
    transition_time_stand_to_sit = 0
    transition_state = 'sitting'
    total_time_sitting = 0
    total_time_standing = 0
    total_time_walking = 0
    last_state = 'sitting'

    prev_trunk_angle = 0
    trunk_velocity = 0

    keypoint_ids = {0:'Nose', 1:'Left Eye', 2:'Right Eye', 3:'Left Ear', 4:'Right Ear', 5:'Left Shoulder', 6:'Right Shoulder', 7:'Left Elbow', 8:'Right Elbow', 9:'Left Wrist', 10:'Right Wrist', 11:'Left Hip', 12:'Right Hip', 13:'Left Knee', 14:'Right Knee', 15:'Left Ankle', 16:'Right Ankle'}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        if results is None:
            print("No objects detected in the frame.")
            continue  # Skip frames where no objects are detected

        confidence = results[0].keypoints.conf[0].numpy()
        keypoints = results[0].keypoints.xy[0].numpy()

        count = 0
        for idx, point in enumerate(keypoints):
            x, y = map(int, point)
            keypoints_df = keypoints_df._append({'video_name': video_file, 'frame_number': cap.get(cv2.CAP_PROP_POS_FRAMES), 'keypoint': keypoint_ids[int(idx)], 'x_coordinate': x, 'y_coordinate': y, 'visibility_score':confidence[int(idx)]}, ignore_index=True)

            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            if count == 5:
                left_shoulder = [x,y]
            elif count == 6:
                right_shoulder = [x,y]
            elif count == 11:
                left_hip = [x,y]
            elif count == 12:
                right_hip = [x,y]
            elif count == 13:
                left_knee = [x,y]
            elif count == 14:
                right_knee = [x,y]
            elif count == 15:
                left_ankle = [x,y]
            elif count == 16:
                right_ankle = [x,y]
            count += 1
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

        distance_threshold = 50
        distance_ankles = np.sqrt((left_ankle[0] - right_ankle[0])**2 + (left_ankle[1] - right_ankle[1])**2)

        angle_left_knee = calculate_angle(left_hip, left_knee, left_ankle)
        angle_left_knee = round(angle_left_knee,2)

        angle_right_knee = calculate_angle(right_hip, right_knee, right_ankle)
        angle_right_knee = round(angle_right_knee, 2)

        angle_left_hip = calculate_angle(left_shoulder, left_hip, left_knee)
        angle_left_hip = round(angle_left_hip, 2)

        angle_right_hip = calculate_angle(right_shoulder, right_hip, right_knee)
        angle_right_hip = round(angle_right_hip, 2)

        left_hip_angle = 180-angle_left_hip
        left_knee_angle = 180-angle_left_knee

        right_hip_angle = 180-angle_right_hip
        right_knee_angle = 180-angle_right_knee

        trunk_angle_left = calculate_angle(left_shoulder, left_hip, left_knee)
        trunk_angle_right = calculate_angle(right_shoulder, right_hip, right_knee)

        trunk_angle = (trunk_angle_left + trunk_angle_right)/2
        trunk_angle = round(trunk_angle, 2)
        print("Trunk Angle:", trunk_angle)

        trunk_acceleration = round(abs(trunk_angle - prev_trunk_angle), 2)
        prev_trunk_angle = trunk_angle

        cv2.putText(frame, f"Trunk Angle: {trunk_angle}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Trunk Acceleration: {trunk_acceleration}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if 105 < trunk_angle < 150:
            if trunk_angle > previous_angle:
                transition_state = "sit to stand"
                transition_time_sit_to_stand += 1 / fps
                log_entry = {'video name': video_file, 'frame number': cap.get(cv2.CAP_PROP_POS_FRAMES), 'transition': transition_state, 'trunk angle': trunk_angle, 'trunk acceleration': trunk_acceleration}
                log_df = log_df._append(log_entry, ignore_index=True)
                last_state = transition_state
            elif trunk_angle < previous_angle:
                transition_state = "stand to sit"
                transition_time_stand_to_sit+= 1 / fps
                log_entry = {'video name': video_file, 'frame number': cap.get(cv2.CAP_PROP_POS_FRAMES), 'transition': transition_state, 'trunk angle': trunk_angle, 'trunk acceleration': trunk_acceleration}
                log_df = log_df._append(log_entry, ignore_index=True)
                last_state = transition_state

        elif 90 < trunk_angle < 105:
            if last_state == "stand to sit":
                if distance_ankles < distance_threshold:
                    transition_state = "standing"
                else:
                    transition_state = "walking"
            else:
                transition_state = "sitting"

        elif 150 < trunk_angle < 180:
            if distance_ankles < distance_threshold:
                transition_state = "standing"
            else:
                transition_state = "walking"
            last_state = transition_state

        elif trunk_angle < 90:
            if transition_state == "sit to stand" or transition_state == "sitting":
                transition_time_sit_to_stand += 1 / fps
                log_entry = {'video name': video_file, 'frame number': cap.get(cv2.CAP_PROP_POS_FRAMES), 'transition': "sit to stand", 'trunk angle': trunk_angle, 'trunk acceleration': trunk_acceleration}
                log_df = log_df._append(log_entry, ignore_index=True)
                last_state = transition_state
            elif transition_state == "stand to sit" or transition_state == "standing":
                transition_time_stand_to_sit += 1 / fps
                log_entry = {'video name': video_file, 'frame number': cap.get(cv2.CAP_PROP_POS_FRAMES), 'transition': "stand to sit", 'trunk angle': trunk_angle, 'trunk acceleration': trunk_acceleration}
                log_df = log_df._append(log_entry, ignore_index=True)
                last_state = transition_state

        previous_angle = trunk_angle

        if transition_state == 'standing' or transition_state == 'stand to sit':
            total_time_standing += 1 / fps

        if transition_state == 'walking':
            total_time_walking += 1 / fps

        if transition_state == 'sitting' or  transition_state == 'sit to stand':
            total_time_sitting += 1 / fps

        cv2.putText(frame, f"Total time spent standing: {round(total_time_standing, 2)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Total time spent sitting: {round(total_time_sitting, 2)}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Total time spent walking: {round(total_time_walking, 2)}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Transition time to sit: {round(transition_time_stand_to_sit, 2)} sec", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Transition time to stand: {round(transition_time_sit_to_stand, 2)} sec", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        output_video.write(frame)

excel_file_path = "C:/Users/white/logfile.xlsx"
log_df.to_excel(excel_file_path, index=False)

excel_file_path_keypoints = "C:/Users/white/keypoints.xlsx"
keypoints_df.to_excel(excel_file_path_keypoints, index=False)
cap.release()
output_video.release()
